In [1]:
# 1.週成交量大於千張；
# 2.5日、10日、20日均線糾結，且糾結幅度要在5％以內；
# 3.主力連買進3天以上，且張數必須大於300張；
# 4.股價需要在月線之上
# 最後，若搭配股票千張股東人數增加、百張股東人數減少，表示大戶正從市場吸收籌碼，在散戶賣股後，後續噴漲機會大。

## 開始

In [2]:
import finlab

In [3]:
finlab.login(api_token='ID0hv1JVLP3bs86Qg/xslT1fwKjN0bclQVRCH7eQiAYh3e5/XNh77u3zaPDOOGJQ#vip_m')

輸入成功!


In [4]:
from finlab import data
from finlab.backtest import sim
import pandas as pd
import numpy as np

In [5]:
benchmark = data.get('benchmark_return:發行量加權股價報酬指數', save_to_storage=True)
close = data.get('etl:adj_close', save_to_storage=True)

volume = data.get('price:成交股數', save_to_storage=True)
volume_week = volume.rolling(5).mean()
ma_5 =  close.average(5)
ma_10 =  close.average(10)
ma_20 =  close.average(20)


 Use "pip install finlab==0.4.4" to update the latest version.


In [6]:
vol=data.get('price:成交股數')
close = data.get('price:收盤價')
pe = data.get('price_earning_ratio:本益比')
rev=data.get('monthly_revenue:當月營收')
rev_rf=data.get('monthly_revenue:去年同月增減(%)')
rev_YoY=data.get('monthly_revenue:去年同月增減(%)')
rev_ma3=rev.average(3)
rev_ma12=rev.average(12)
vol_ma5 = vol.average(5)

In [7]:
data.get('broker_mapping')[data.get('broker_mapping')['name']=='台灣摩根士丹利']

,broker,name
391,391,台灣摩根士丹利


In [8]:
chips_15 = data.get('broker_transactions', save_to_storage=True)
#大摩391

In [9]:
chips_bigmo = chips_15[chips_15['broker'] == 391.0].dropna()

In [10]:
chips_bigmo

,stock_id,broker,buy,sell
date,,,,
2018-01-05,0050,391.0,36.0,0.0
2018-01-12,0050,391.0,41.0,82.0
2018-01-29,0050,391.0,44.0,0.0
2018-02-02,0050,391.0,0.0,31.0
2018-02-07,0050,391.0,256.0,0.0
...,...,...,...,...
2023-05-17,9962,391.0,3.0,24.0
2023-05-18,9962,391.0,8.0,20.0
2023-05-19,9962,391.0,0.0,63.0


In [11]:
chips_bigmo['surplus'] = chips_bigmo['buy'] - chips_bigmo['sell']
chips_bigmo

,stock_id,broker,buy,sell,surplus
date,,,,,
2018-01-05,0050,391.0,36.0,0.0,36.0
2018-01-12,0050,391.0,41.0,82.0,-41.0
2018-01-29,0050,391.0,44.0,0.0,44.0
2018-02-02,0050,391.0,0.0,31.0,-31.0
2018-02-07,0050,391.0,256.0,0.0,256.0
...,...,...,...,...,...
2023-05-17,9962,391.0,3.0,24.0,-21.0
2023-05-18,9962,391.0,8.0,20.0,-12.0
2023-05-19,9962,391.0,0.0,63.0,-63.0


In [12]:
main_force = pd.DataFrame(data={'surplus':chips_bigmo.reset_index()['surplus'].to_numpy()}, index=[chips_bigmo.reset_index()['date'], chips_bigmo.reset_index()['stock_id']])

main_force = main_force.reset_index(level=1)
main_force = main_force.pivot_table(values='surplus', index=main_force.index, columns='stock_id')
main_force

stock_id,0050,0056,006205,006207,00631L,00632R,00633L,00639,00663L,00664R,...,9941A,9942,9943,9944,9945,9946,9951,9955,9958,9962
date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,249.0,NaN,NaN,NaN,102.0,NaN
2018-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,434.0,NaN,NaN,NaN,219.0,NaN
2018-01-05,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,329.0,NaN,NaN,NaN,379.0,NaN
2018-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-73.0,NaN,NaN,NaN,-108.0,-12.0
2023-05-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,-206.0,-63.0
2023-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,88.0,18.0,NaN,-76.0,NaN,NaN,NaN,-63.0,-19.0


In [13]:
from finlab import data
from finlab.backtest import sim

close = data.get("price:收盤價")
vol = data.get("price:成交股數")
vol_ma = vol.average(10)
rev = data.get('monthly_revenue:當月營收')
rev_year_growth = data.get('monthly_revenue:去年同月增減(%)')
rev_month_growth = 	data.get('monthly_revenue:上月比較增減(%)')



In [14]:
score = data.get('etl:finlab_tw_stock_market_ind')['score']


In [26]:
# 股價創年新高
cond1 = (close == close.rolling(250).max())

# 排除月營收連3月衰退10%以上
cond2 = ~(rev_year_growth < -10).sustain(3) 

# 排除月營收成長趨勢過老(12個月內有至少8個月單月營收年增率大於60%)
cond3 = ~(rev_year_growth > 60).sustain(12,8)

# 確認營收底部，近月營收脫離近年谷底(連續3月的"單月營收近12月最小值/近月營收" < 0.8)
cond4 = ((rev.rolling(12).min())/(rev) < 0.8).sustain(3)

# 單月營收月增率連續3月大於-40%
cond5 = (rev_month_growth > -40).sustain(3)

# 流動性條件
cond6 = vol_ma > 1000*1000

condition4 = (close != close.rolling(5).max()).sustain(5,2)
condition3 = score > 3
condition7 = ((main_force*1000)/volume) > 0.2
condition8 = main_force.shift(1).rolling(5).sum() < 0

buy = cond4  * condition7
#  * cond6 * condition4
# 買比較冷門的股票
buy = vol_ma*buy
# buy = buy[buy>0]
# buy = buy.is_smallest(5)
position = buy
position = position[position.index >= '2018-09-01']
report = sim(position=position , resample="2D", upload=True, position_limit=1/3, fee_ratio=1.425/1000/5, stop_loss=0.05,take_profit=0.2,name='大摩倒貨', live_performance_start='2022-05-01', trade_at_price='close',mae_mfe_window=5)
report.display()

--Stats--
annualized_rate_of_return    -27.14%
sharpe                         -2.46
max_drawdown                  -72.8%
win_ratio                      43.5%

--Latest Position--


2889    0.333333
6276    0.300140
Name: 2023-05-24 00:00:00, dtype: float64

In [17]:
report.display_mae_mfe_analysis()

In [18]:
report.trades['return'].max()

0.15250965250965254

In [19]:
# condition1 = ((main_force*1000)/volume) > 0.05
# condition2 = main_force.shift(1).rolling(5).sum() < 0
# condition4 = (close != close.rolling(5).max()).sustain(5,2)
# extra_conditions =  ((rev_ma3 > rev_ma12).sustain(3)) & (rev_rf >= 20) & (rev_ma3/rev_ma3.shift(12) >= 1.15) & (vol_ma5 >= 100*1000)
# conditions =  condition1 * condition2 

In [20]:
# position= conditions
# position = position
# # position = position.is_smallest(10).reindex(rev.index_str_to_date().index, method='ffill')

In [21]:
# position

In [22]:
# report = sim(position, position_limit=0.2,resample='2W',stop_loss=0.2, live_performance_start='2022-10-01')
# report.display()

In [23]:
# report.display_mae_mfe_analysis()